Database Credentials


In [27]:
import os

from dotenv import load_dotenv

load_dotenv()

True

In [28]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [37]:
import os
from dotenv import load_dotenv
from langchain_community.utilities.sql_database import SQLDatabase

# Load environment variables
load_dotenv()

# Get database credentials from environment variables
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")

# Create database connection
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")
print(db.dialect)
print(db.get_usable_table_names())
print(db.table_info)

mysql
['ad_creatives', 'ad_insights', 'ad_tracking_specs', 'ads', 'campaign_actions', 'campaign_conversions', 'campaign_insights', 'campaigns']

CREATE TABLE ad_creatives (
	id BIGINT NOT NULL, 
	name VARCHAR(255), 
	title VARCHAR(255), 
	body TEXT, 
	thumbnail_url TEXT, 
	video_id BIGINT, 
	call_to_action_type VARCHAR(50), 
	page_id VARCHAR(50), 
	PRIMARY KEY (id)
)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from ad_creatives table:
id	name	title	body	thumbnail_url	video_id	call_to_action_type	page_id
120213321698260311	Be the Monk in Marketing 2024-05-22-06a6e70c8d6b902539eaa0f241f2e1c4	Be the Monk in Marketing	🚀 Elevate your game! Dentsu & Publicis insiders swear by us. Unlock 🔓 top case studies for free.	https://external.fmaa1-2.fna.fbcdn.net/emg1/v/t13/6227554188890667404?url=https%3A%2F%2Fwww.facebook	None	SUBSCRIBE	125267114004054
120213858232380311	{{product.name}} 2024-06-10-e966e3395993c4a04c6f1e2f176eb43f	None	None	https://scontent.fmaa1-2.fna

In [30]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
generate_query_chain=create_sql_query_chain(llm, db)

query=generate_query_chain.invoke({"question":"which campaign has the most clicks?"})

print(query)


SELECT campaigns.id, campaigns.name, SUM(campaign_insights.clicks) AS total_clicks
FROM campaigns
JOIN campaign_insights ON campaigns.id = campaign_insights.campaign_id
GROUP BY campaigns.id
ORDER BY total_clicks DESC
LIMIT 1;


In [31]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
execute_query = QuerySQLDataBaseTool(db=db)
execute_query.invoke(query)


"[(120214454200140311, 'MM_Motion(B)_VideoAd_07072024', Decimal('2122'))]"

In [32]:
chain= generate_query_chain | execute_query
chain.invoke({"question":"Which campaign has the most clicks?"})

"[(120214454200140311, 'MM_Motion(B)_VideoAd_07072024', Decimal('2122'))]"

In [33]:
chain.get_prompts()[0].pretty_print()

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the S

In [35]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
     """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

 Question: {question}
 SQL Query: {query}
 SQL Result: {result}
 Answer: """
 )

rephrase_answer = answer_prompt | llm | StrOutputParser()

chain = (
    RunnablePassthrough.assign(query=generate_query_chain).assign(
        result=itemgetter("query") | execute_query
     )
     | rephrase_answer
 )

chain.invoke({"question": "Campaign with the most impressions?"})








"The campaign with the most impressions is 'MM_UGC_VideoAd_20062024' with 213,083 impressions."